In [8]:
import os
import sys
from datetime import datetime, timedelta,date
from multiprocessing import Pool

import numpy as np
from scipy import stats 
from matplotlib import pyplot as plt
import pandas as pd
import xarray as xr
import rioxarray as rio
import rasterio

from tqdm import tqdm

In [9]:
PROJECT_DIR = 'E:\\project_work'

# input files
DATA_DIR = os.path.join(PROJECT_DIR, 'data')
HDF_FILE_DIR = os.path.join(DATA_DIR, 'hdf5')

# output files
OUTPUT_DIR = os.path.join(PROJECT_DIR, 'output')
NETCDF_DIR = os.path.join(OUTPUT_DIR, 'netcdf')
YEALY_DATA_DIR=os.path.join(NETCDF_DIR, 'yearly')



os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(NETCDF_DIR, exist_ok=True)

HDF_FILE_LIST = [os.path.join(HDF_FILE_DIR, file) for file in os.listdir(HDF_FILE_DIR) if file.endswith('.hdf')]





In [10]:
def separate_var(file, var_list = None):
    if var_list is None:
        var_list = ['LST_Day_CMG', 'LST_Night_CMG', 'QC_Night', 'QC_Day']
    data = xr.open_dataset(file, engine='rasterio')
    ds = xr.Dataset()
    for var in var_list:
        ds[var] = data[var].copy()
    data = None
    return ds

def get_date(day_of_year, year):
    return datetime(year, 1, 1) + timedelta(days=day_of_year - 1)


def get_month_start(year):
    def get_two_place_representation(month):
        return str(month) if month>9 else f'0{month}'
    
    month_start = []
    for mon in range(1, 13):
        month_start.append(int(datetime.strftime(
            datetime.fromisoformat(f'{year}-{get_two_place_representation(mon)}-01'),
            '%j')))
    month_start.append(int(datetime.strftime(datetime.fromisoformat(f'{year}-12-31'),'%j'))+1)
    
    return month_start

def read_monthly_files(year, month, month_start=None, file_list=HDF_FILE_LIST, var_list=None, sel_lat=(8,38), sel_lon=(98, 67)):
    if var_list is None:
        var_list = ['LST_Day_CMG', 'LST_Night_CMG', 'QC_Night', 'QC_Day']
      
    if month_start is None:
        month_start = get_month_start(year)
        
    current_mon_start = month_start[month-1]
    next_mon_start   = month_start[month]
    
    no_of_file_in_month=0
    for file in file_list:
        if year == int(os.path.basename(file)[9:13]):         
            if (int(os.path.basename(file)[13:16]) >= current_mon_start) and (int(os.path.basename(file)[13:16]) < next_mon_start):
                    no_of_file_in_month+=1
                    
        
        
    current_month_files = sorted([file 
                                  for file in file_list 
                                  if year == int(os.path.basename(file)[9:13])         
                                  if (int(os.path.basename(file)[13:16]) >= current_mon_start) and (int(os.path.basename(file)[13:16]) < next_mon_start)
                                 ])
    if len(current_month_files) == 0:
        return None
    
    # import pdb; pdb.set_trace() 
    data = xr.open_dataset(file_list[0], engine='rasterio').sel(x=slice(*sel_lat),y=slice(*sel_lon))
    # import pdb; pdb.set_trace()
    data_shape = data['LST_Day_CMG'].values.squeeze().transpose().shape
    data = None
    # import pdb; pdb.set_trace()
    
    
    data_for_month = np.zeros([*data_shape, len(var_list), no_of_file_in_month])
    count = 0
    time = []
    
    # import pdb; pdb.set_trace()
    for file in current_month_files:
        time.append(get_date(day_of_year = int(os.path.basename(file)[13:16]),
                             year = int(os.path.basename(file)[9:13])))
        data = xr.open_dataset(file, engine='rasterio').sel(x=slice(*sel_lat),y=slice(*sel_lon))
            
        _current_date_data = np.array([data[var_name].values.squeeze().transpose()
                                      for var_name in var_list])
        current_date_data = np.moveaxis(_current_date_data, [0], [2])
        #import pdb; pdb.set_trace()
        
        data_for_month[:, :, :, count] = current_date_data
        count += 1
    # import pdb; pdb.set_trace()     
    data_attrs = {'ALGORITHMPACKAGENAME': 'MOD_PR11C1',
                  'ALGORITHMPACKAGEVERSION': 6,
                  'ASSOCIATEDINSTRUMENTSHORTNAME.1': 'MODIS',
 'ASSOCIATEDPLATFORMSHORTNAME.1': 'Terra',
 'ASSOCIATEDSENSORSHORTNAME.1': 'MODIS',
 'AUTOMATICQUALITYFLAG.1': 'Passed',
 'AUTOMATICQUALITYFLAGEXPLANATION.1': 'No automatic quality assessment is performed in the PGE.',
 'CLOUD_CONTAMINATED_LST_SCREENED': 'YES',
 'DAYNIGHTFLAG': 'Both',
 'DESCRREVISION': 6.1,
 'EASTBOUNDINGCOORDINATE': 180.0,
 'GLOBALGRIDCOLUMNS': 7200,
 'GLOBALGRIDROWS': 3600,
 'HDFEOSVersion': 'HDFEOS_V2.19',
 'identifier_product_doi': '10.5067/MODIS/MOD11C1.061',
 'INSTRUMENTNAME': 'Moderate-Resolution Imaging Spectroradiometer',
 'LOCALGRANULEID': 'MOD11C1.A2018147.061.2021334232853.hdf',
 'LOCALVERSIONID': '6.3.0',
 'LONGNAME': 'MODIS/Terra Land-Surface Temperature/Emissivity Daily Global 0.05Deg CMG',
 'NORTHBOUNDINGCOORDINATE': 90.0,
 'PARAMETERNAME.1': 'MOD DAILY CMG 3MIN LST',
 'PGEVERSION': '6.4.2',
 'PLATFORMSHORTNAME': 'Terra',
 'PROCESSINGCENTER': 'MODAPS',
 'PROCESSINGDATETIME': '2021-11-30T23:28:53.000Z',
 'PROCESSINGENVIRONMENT': 'Linux minion7473 3.10.0-1160.42.2.el7.x86_64 #1 SMP Tue Sep 7 14:49:57 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux',
    }
    ds = xr.Dataset(data_vars={var_name: xr.DataArray(data_for_month[:, :, i, :].squeeze(), 
                                                      name=var_name, 
                                                      dims=('lon', 'lat', 'time'), 
                                                      coords={'lon': data['x'].values, 'lat': data['y'].values, 'time': time},
                                                      attrs = data[var_name].attrs,
                                                      ) 
                               for i, var_name in enumerate(var_list)},
                    attrs = data_attrs,
                   )
    return ds

for year in range(2022,2023):
    for month in range(1,12+1):
        ds = read_monthly_files(year, month)
        if ds is not None: 
            ds.to_netcdf(os.path.join(YEALY_DATA_DIR, f'MOD11C1.061_{year}_{month}.nc'), 
                     encoding={'QC_Day': {'dtype': 'int16', 'zlib': True, 'complevel': 9, '_FillValue':-999}, 
                           'QC_Night': {'dtype': 'int16', 'zlib': True, 'complevel': 9, '_FillValue':-999},
                           'LST_Day_CMG':  {'dtype': 'int16', 'scale_factor': 0.02, 'zlib': True, 'complevel': 9, '_FillValue':-999},
                           'LST_Night_CMG':  {'dtype': 'int16', 'scale_factor': 0.02, 'zlib': True, 'complevel': 9, '_FillValue':-999},}
                           )



KeyboardInterrupt: 

In [ ]:
def save_nc_for_year(year):
    print(f'processing {year}...')
    for month in range(1, 12 + 1):
        ds = read_monthly_files(year, month)
        if ds is not None:
            ds.to_netcdf(os.path.join(NETCDF_DIR, f'MOD11C1.061_{year}_{month}.nc'), 
                     encoding={'QC_Day': {'dtype': 'int16', 'zlib': True, 'complevel': 9, '_FillValue':-999}, 
                           'QC_Night': {'dtype': 'int16', 'zlib': True, 'complevel': 9, '_FillValue':-999},
                           'LST_Day_CMG':  {'dtype': 'int16', 'scale_factor': 0.02, 'zlib': True, 'complevel': 9, '_FillValue':-999},
                           'LST_Night_CMG':  {'dtype': 'int16', 'scale_factor': 0.02, 'zlib': True, 'complevel': 9, '_FillValue':-999},}
                           )
    
with Pool() as pool:
    res = pool.map(save_nc_for_year, list(range(2002, 2021 + 1)))